In [23]:
#importem el dataset
import dill
dill.load_session('data.db')

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import plot_importance
from sklearn.ensemble import StackingClassifier
from sklearn.base import clone
from scipy.stats import uniform, randint

In [25]:
#Definim train i test, estandaritzant les dades
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10, test_size = 0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
model1 = OneVsOneClassifier(LogisticRegression())
model1.fit(X_train, y_train)

OneVsOneClassifier(estimator=LogisticRegression())

In [27]:
model2 = OneVsRestClassifier(LogisticRegression())
model2.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [28]:
model3 = OutputCodeClassifier(LogisticRegression())
model3.fit(X_train, y_train)

OutputCodeClassifier(estimator=LogisticRegression())

In [29]:
#Cerca de millors hiperparàmetres per KNN
accuracy=[]
accuracy_w=[]
n_neighbors=[]
for i in range(1,50,2):
    knn=neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    accuracy.append(knn.score(X_test, y_test))
    knn=neighbors.KNeighborsClassifier(n_neighbors=i,weights="distance")
    knn.fit(X_train, y_train)
    accuracy_w.append(knn.score(X_test, y_test))
    n_neighbors.append(i)
model4=neighbors.KNeighborsClassifier(n_neighbors=n_neighbors[accuracy.index(max(accuracy))]).fit(X_train,y_train)
model5=neighbors.KNeighborsClassifier(n_neighbors=n_neighbors[accuracy_w.index(max(accuracy_w))],weights="distance").fit(X_train,y_train)

In [30]:
model6 = DecisionTreeClassifier()
model6.fit(X_train, y_train)

DecisionTreeClassifier()

In [31]:
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 16),
              "min_samples_leaf": randint(1, 100),
              "criterion": ["gini", "entropy"],
              "splitter": ["best", "random"]}
dt = DecisionTreeClassifier()
dt_random = RandomizedSearchCV(estimator = dt, param_distributions = param_dist, n_iter = 2000, cv = 3, verbose=2, n_jobs = -1)
dt_random.fit(X_train, y_train)
model7=dt_random.best_estimator_

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits


C:\Users\Bernat\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.91421229 0.91351756 0.91393445 ... 0.91185038 0.8901086  0.91421229]
  warnings.warn(


In [32]:
model8 = RandomForestClassifier(n_estimators=500)
model8.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500)

In [33]:
param_grid = { 
    'n_estimators': [200,500,1000],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [5,10,20],
    'criterion' :['gini', 'entropy']
}

rf = RandomForestClassifier()
rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3)
rf_random.fit(X_train, y_train)
model9 = rf_random.best_estimator_

In [34]:
model10 = xgb.XGBClassifier(objective="multi:softprob")
model10.fit(X_train,y_train,eval_metric='merror')

C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [35]:
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3),
    "max_depth": randint(2, 6), 
    "n_estimators": randint(100, 150),
    "subsample": uniform(0.6, 0.4),
    "objective":"multi:softprob"
}
xgb_model=xgb.XGBClassifier()
xgb_random = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=150, cv=3, verbose=1, n_jobs=1, return_train_score=True)
xgb_random.fit(X_test, y_test,eval_metric='merror')
model11= xgb_random.best_estimator_

Fitting 3 folds for each of 150 candidates, totalling 450 fits


C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use 

In [36]:
level0 = list()
level0.append(('knn', clone(model4)))
level0.append(('rf', clone(model9)))
level0.append(('ovo', clone(model1)))
level0.append(('xgb', clone(model11)))
level1 = LogisticRegression(max_iter=10000)
model12 = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
model12.fit(X_train,y_train)

C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:12:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:18:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:19:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Bernat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:19:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


StackingClassifier(cv=5,
                   estimators=[('knn', KNeighborsClassifier(n_neighbors=9)),
                               ('rf',
                                RandomForestClassifier(criterion='entropy',
                                                       max_depth=20,
                                                       n_estimators=1000)),
                               ('ovo',
                                OneVsOneClassifier(estimator=LogisticRegression())),
                               ('xgb',
                                XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=0.8402119883766755,
                                              e...
                                              max_delta_step=0, max_depth=4,
                                              min_ch

In [37]:
dill.dump_session('fittedmodels.db')